# Instalando o **Gdown** 

In [ ]:
!sudo apt-get install pearl

!sudo apt-get install git -y

!git clone https://github.com/circulosmeos/gdown.pl.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package pearl
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
Cloning into 'gdown.pl'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 95 (delta 0), reused 1 (delta 0), pack-reused 92
Unpacking objects: 100% (95/95), done.


# **Utilizando o Gdown para o download do dataset zipado e extraindo seus itens**

In [ ]:
#!./gdown.pl/gdown.pl https://drive.google.com/file/d/1jbWJBhNFST3yJR_9sXuYGK-YWki0Y6gi/view?usp=sharing arq.zip
#!./gdown.pl/gdown.pl https://drive.google.com/file/d/1T2fq8utS-TRHckYGw4AtG-H4aoJlU-Lj/view?usp=sharing arq.zip
!./gdown.pl/gdown.pl https://drive.google.com/file/d/15YLtDg4lNxSewgajz3dU3jOt5luZeCpF/view?usp=sharing arq.zip
!unzip arq.zip
!clear

# Informações da **GPU**

In [ ]:
#GPU INFO
import tensorflow as tf
import keras
import sys
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
print("Versão Python: " + sys.version)
print("Versão Tensorflow: " + tf.__version__)
print("Versão Keras: " + keras.__version__)

Sun Feb 14 18:02:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Criação das Redes

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from keras.layers.normalization import BatchNormalization
from keras.models import Model
import tensorflow as tf
import keras

def vgg16():
  from keras.applications.vgg16 import VGG16
  import keras
  from keras.models import Model
  from keras.layers import Dense
  from keras import optimizers
  vggmodel = VGG16(
      include_top=True,
      weights=None,
      input_tensor=None,
      input_shape=(299,299, 3),
      pooling=None,
      classes=2,
      classifier_activation="softmax",
  )
  return vggmodel

def inceptionv3():
  
  model = keras.applications.InceptionV3(include_top=True, weights='imagenet')
  x = Dense(3, activation='softmax', name='predictions')(model.layers[-2].output)
  my_model = Model(model.input, x)
  return my_model

def resnet152():
  # Generate a model with all layers (with top)
  resnet = keras.applications.ResNet152(weights='imagenet', include_top=True)
  #resnet.summary()
  #Add a layer where input is the output of the  second last layer 
  x = Dense(3, activation='softmax', name='predictions')(resnet.layers[-2].output)

  #Then create the corresponding model 
  my_model = Model(resnet.input, x)
  return my_model


model = inceptionv3()
SaveRede = "IncepctionMini.h5"
target_shape = (299,299)
#model.summary()

96116736/96112376 [==============================] - 1s 0us/step


In [ ]:
import keras
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

# **Treinamento**




In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

checkpoint = ModelCheckpoint("drive/MyDrive/NewSaves/" + SaveRede, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')



BATCH_SIZE = 16

trdata = ImageDataGenerator(rescale=1. / 255)
#train_generator = trdata.flow_from_directory(directory="train",target_size=target_shape, batch_size=BATCH_SIZE)
tsdata = ImageDataGenerator(rescale=1. / 255)
#validation_generator = tsdata.flow_from_directory(directory="test", target_size=target_shape, batch_size=BATCH_SIZE)

#hist = model.fit(train_generator, validation_data=validation_generator, callbacks=[checkpoint], epochs=10)

#np.save('drive/MyDrive/NewSaves/' + SaveRede +'.npy', hist.history) #salvar o history da rede para plotar depois

# **Teste de Rede**

In [ ]:
import numpy as np
import os
#metodo para classificação
def classificar(path, model_, shape):
  from keras.preprocessing import image
  imagem_teste = image.load_img( path, target_size = shape) #target_size para redimencionar imagem para o tamanho que a rede exige

  imagem_teste = image.img_to_array(imagem_teste)
  imagem_teste /= 255 #dividindo numero para o tensorflow

  imagem_teste = np.expand_dims(imagem_teste, axis = 0) #adicionando shape para padrões de rede

  import os
  previsao = model_.predict(imagem_teste) #predict imagem
  saida = os.sys.maxsize * -1 #minimo valor possivel do sistema para a comparação da saida da rede
  prev = []
  prev.append(previsao[0][0])
  prev.append(previsao[0][1])
  prev.append(previsao[0][2])
  previsao = previsao[0] #transformando lista de lista em apenas uma lista
  pos = 0
  for x in range(len(previsao)):
    if saida < previsao[x]:
      saida = previsao[x]
      pos = x
    
  return saida, pos, prev




def testarRede(model_, shape):
  #path = "Covid19-dataset/train/"
  path = "test/"
  arr = sorted(os.listdir(path))

  covid = {"COVID": 0, "Viral Pneumonia": 0, "NORMAL": 0}
  pneumo = {"COVID": 0, "Viral Pneumonia": 0, "NORMAL": 0}
  normal = {"COVID": 0, "Viral Pneumonia": 0, "NORMAL": 0}

  listaResp = []
  y_correto = []
  y_predict = []
  acertos = 0
  total = 0

  for x in arr:
    if (x != '.ipynb_checkpoints'):
      listaResp.append(x)
  print(listaResp)
  for x in arr:
    imagens = os.listdir(path + x)
    for y in imagens:
      # x(nome da pasta) | y(nome do arquivo)
      #print("Pasta: " + x + " | " + y)

      #classificar
      porcentagem, posicaoResp, previsao = classificar(path + x + "/" + y, model_, shape)
      if(x == "COVID"):
        covid[listaResp[posicaoResp]]+=1
        y_correto.append(0)
      elif(x == "Viral Pneumonia"):
        pneumo[listaResp[posicaoResp]]+=1
        y_correto.append(1)
      elif(x == "NORMAL"):
        normal[listaResp[posicaoResp]]+=1
        y_correto.append(2)
      
      #criando listas de resposta
      
      y_predict.append(previsao)
      print(f"{str(porcentagem):10}  |  Correto: {x:18} | Rede Model: {listaResp[posicaoResp]} | Nome Imagem: {y}")
      #print("#", end='')
      if (x == listaResp[posicaoResp]):
        acertos+=1
      total+=1




  print(covid)
  print(pneumo)
  print(normal)
  porcentagem = acertos*100/total
  print(f"Acertos: {acertos} Total: {total} Porcentagem: {porcentagem}")
  return (acertos, total, porcentagem), covid, pneumo, normal, y_correto, y_predict

In [ ]:
model = inceptionv3()
model.load_weights("drive/MyDrive/NewSaves/InceptionV3Colab64Batch250EpochsTransfer.h5")
tupla, covid, pneumo, normal, y_correto, y_predict = testarRede(model, (299,299))
print(y_correto)
print(y_predict)

['COVID', 'NORMAL', 'Viral Pneumonia']
0.9999876   |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 025.jpeg
0.99933785  |  Correto: COVID              | Rede Model: COVID | Nome Imagem: COVID-00036.jpg
0.9957086   |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 01.jpeg
0.9999993   |  Correto: COVID              | Rede Model: COVID | Nome Imagem: COVID-00001.jpg
0.7967516   |  Correto: COVID              | Rede Model: COVID | Nome Imagem: auntminnie-2020_01_31_20_24_2322_2020_01_31_x-ray_coronavirus_US.jpg
0.9999982   |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 078.jpeg
0.99999535  |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 083.jpeg
0.9993917   |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 027.jpeg
0.99999034  |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 07.jpg
0.99999714  |  Correto: COVID              | Rede Model: COVID | Nome Imagem: 0115.jpeg
0.99929094  |  Corr